In [2]:
import cv2  
import math 

from ultralytics import YOLO

In [3]:
# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO("yolo11n.pt")  # Contenedores

# Etiqueta de las distintas clases
classNames = [
    "person",
    "bus",
    "car",
    "motorbike",
    "truck"
]

# Cargar el video desde archivo en lugar de webcam
filename = "prueba.mp4"  # Nombre del archivo de video
vid = cv2.VideoCapture(filename)

new_width = 1080
new_height = 720
desplazamiento = 200

# Bucle para procesar cada fotograma del video
while True:
    # Leer fotograma por fotograma
    ret, img = vid.read()

    # Si no hay más fotogramas, salir del bucle
    if not ret:
        print("Video finalizado")
        break
    
    img_resized = cv2.resize(img, (new_width, new_height))


    # Detecta en el fotograma
    results = model.track(img_resized[desplazamiento:720, 0:1080], stream=True, persist=True, classes=[1,2,3,4,5])

    # Para cada detección
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Contenedor
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = (
                int(x1),
                int(y1)+desplazamiento,
                int(x2),
                int(y2)+desplazamiento,
            )  # Convertir a valores enteros

            # Confianza
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confianza --->", confidence)

            # Clase
            cls = int(box.cls[0])
            track_id = str(int(box.id[0].tolist())) if box.id is not None else ' '
            print("ID --->", track_id)

            # Verificar si el índice de clase está dentro del rango de `classNames`
            if cls < len(classNames):
                print("Clase -->", classNames[cls])
            else:
                print(f"Clase con índice {cls} no está definida en classNames.")
                continue


            # Convierte identificador numérico de clase a un color RGB
            escala = int((cls / len(classNames)) * 255 * 3)
            if escala >= 255 * 2:
                R = 255
                G = 255
                B = escala - 255 * 2
            else:
                if escala >= 255:
                    R = 255
                    G = escala - 255
                    B = 0
                else:
                    R = escala
                    G = 0
                    B = 0

            # Dibuja el contenedor y clase
            cv2.rectangle(img_resized, (x1, y1), (x2, y2), (R, G, B), 3)
            cv2.putText(
                img_resized,
                classNames[cls]+ track_id,
                [x1, y1],
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 0, B),
                2,
            )

    # Muestra fotograma procesado
    cv2.imshow("Video", img_resized)

    # Detenemos pulsado ESC o cuando el video termina
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura y cierra ventanas
vid.release()
cv2.destroyAllWindows()



0: 320x640 1 car, 1 bus, 216.1ms
Confianza ---> 0.86
ID ---> 1
Clase con índice 5 no está definida en classNames.
Confianza ---> 0.73
ID ---> 2
Clase --> car
Speed: 17.0ms preprocess, 216.1ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 bus, 74.3ms
Confianza ---> 0.9
ID ---> 1
Clase con índice 5 no está definida en classNames.
Confianza ---> 0.72
ID ---> 2
Clase --> car
Speed: 3.0ms preprocess, 74.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 bus, 73.3ms
Confianza ---> 0.91
ID ---> 1
Clase con índice 5 no está definida en classNames.
Confianza ---> 0.76
ID ---> 2
Clase --> car
Speed: 3.0ms preprocess, 73.3ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 bus, 71.1ms
Confianza ---> 0.9
ID ---> 1
Clase con índice 5 no está definida en classNames.
Confianza ---> 0.77
ID ---> 2
Clase --> car
Speed: 3.0ms preprocess, 71.1ms inference, 1.9ms postprocess per image at sh